In [73]:
import pandas as pd
from pathlib import Path
from collections import defaultdict, Counter
from collections import OrderedDict
import warnings

In [79]:
warnings.simplefilter("ignore", UserWarning)

repo_root = Path().cwd()
print(repo_root)
print(repo_root)

ref_path  = repo_root / "master_sheet_reference_v2.xlsx"
ag_folder    = repo_root / "assessment_guide"
ag_files  = sorted(ag_folder.glob("*.xls*"))

print("Found workbooks:")
for p in ag_files:
    print("•", p.name)

ref_sheets  = pd.read_excel(ref_path,  sheet_name=None)
print("Reference file sheets:", list(ref_sheets.keys()))

c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\lighthouse_dash
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\lighthouse_dash
Found workbooks:
• assessment_guide_diriyah.xlsm
• assessment_guide_qiddiya.xlsm
Reference file sheets: ['P&M Schedule', 'P&M RIsk', 'Cost Estimation & Optimization', 'D&C - Quality', 'D&C - Procurement', 'D&C - HSE & Welfare', 'Innovation & Technology', 'Design & Technical', 'D&C - Construction', 'Strategy & Operations', 'Visuals Menu']


In [80]:
# Unpack reference file
ref_tabs = []     

for sheet_name, df in ref_sheets.items():
    if {"Table", "Column"}.issubset(df.columns): # Check if columns titled "Table" and "Column" exist
        table_cols = (
            df.groupby("Table")["Column"]
              .apply(list)
              .to_dict()
        )
        ref_tabs.append({ sheet_name: table_cols })
    else:
        print(f"Skipping sheet {sheet_name!r}: missing \"Table\" and \"Columm\" columns")

ref_tabs

Skipping sheet 'Visuals Menu': missing "Table" and "Columm" columns


[{'P&M Schedule': {'AddDataPoint(P&M)': ['Additional Data Point',
    'Value',
    'Year',
    'Value Completed (Baseline) in M SAR',
    'Value Completed (Actual) in M SAR',
    'Value Completed (Forecast) in M SAR'],
   'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria', 'Value']}},
 {'P&M RIsk': {'AddDataPoint(P&M)': ['Additional Data Point',
    'Value',
    'Risk Category',
    'Number of Risk Items',
    'Risk',
    'Source',
    'Potential Impact',
    'Risk Rating',
    'Risk Status',
    'Current Measures'],
   'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria',
    'Value',
    'Rating']}},
 {'Cost Estimation & Optimization': {'AddDataPoint(CE&O)': ['Additional Data Point',
    'Value',
    'Year',
    'Budget Value (Baseline) in M SAR',
    'Budget Value (Revised) in M SAR',
    'Budget Value (Forecast) in M SAR'],
   'DevCoAssessmentAnalysis(CE&O)': ['Assessment Criteria', 'Value', 'Rating'],
   'DevCoAssessmentInput(CE&O)': ['Data Point', 'Input Value']}},
 {'D&C -

In [81]:
table_cols = defaultdict(list)

for tab_dict in ref_tabs:
    sheet_map = next(iter(tab_dict.values()))
    for table_name, cols in sheet_map.items():
        table_cols[table_name].extend(cols)
        
for table_name, all_cols in table_cols.items():
    counts = Counter(all_cols)
    dupes = [col for col, n in counts.items() if n > 1]
    if dupes:
        print(f"⚠️ Table “{table_name}” has duplicate columns: {dupes}")
    else:
        print(f"✅ Table “{table_name}” has no repeated columns.")

for table_name, cols in table_cols.items():
    table_cols[table_name] = list(OrderedDict.fromkeys(cols))

print(table_cols)

⚠️ Table “AddDataPoint(P&M)” has duplicate columns: ['Additional Data Point', 'Value']
⚠️ Table “DevCoAssessmentAnalysis(P&M)” has duplicate columns: ['Assessment Criteria', 'Value']
✅ Table “AddDataPoint(CE&O)” has no repeated columns.
✅ Table “DevCoAssessmentAnalysis(CE&O)” has no repeated columns.
✅ Table “DevCoAssessmentInput(CE&O)” has no repeated columns.
⚠️ Table “AddDataPoint(D&C)” has duplicate columns: ['Additional Data Point', 'Value']
⚠️ Table “DevCoAssessmentAnalysis(D&C)” has duplicate columns: ['Assessment Criteria', 'Value', 'Performance Signal Score']
⚠️ Table “DevCoAssessmentInput(D&C)” has duplicate columns: ['Assessment Criteria', 'Data Point', 'Input Value']
✅ Table “AddDataPoint(I&T)” has no repeated columns.
✅ Table “DevCoAssessmentAnalysis(I&T)” has no repeated columns.
✅ Table “DevCoAssessmentAnalysis(D&T)” has no repeated columns.
✅ Table “DevCoAssessmentInput(D&T)” has no repeated columns.
✅ Table “AddDataPoint(S&O)” has no repeated columns.
✅ Table “DevCoAss

In [ ]:
# all_cols = []
# for cols in table_cols.values():
#     all_cols.extend(cols)
# dupe_cols = {c for c, cnt in Counter(all_cols).items() if cnt > 1}

# dupe_cols

{'Additional Data Point',
 'Assessment Criteria',
 'Data Point',
 'Input Value',
 'Performance Signal',
 'Performance Signal Score',
 'Rating',
 'Value',
 'Year'}

In [ ]:
file_frames = []   # to collect df for each file

for data_path in ag_files:
    # derive suffix
    parts = data_path.stem.split("assessment_guide_")
    suffix = parts[1]

    print(f"Reading {data_path.name!r} (suffix = {suffix})")

    pieces = []
    for table_name, cols in table_cols.items():
        df = pd.read_excel(data_path, sheet_name=table_name, header=4)  # pull in AG data. Each table name from column reference sheet is a sheet name in the AG file.
        df.columns = (
            df.columns
            .str.replace(r'\s*\n\s*', ' ', regex=True)  
            .str.strip()                                
        )

        # print(df.columns)

        present = [c for c in cols if c in df.columns] # cols = columns from reference, present = columns in AG
        missing = set(cols) - set(present)
        if missing:
            print(f"⚠️ In table {table_name!r}, missing columns: {missing}")

        df_sub = df[present].copy()
        for c in missing:
            df_sub[c] = pd.NA

        # rename_map = {
        #     col: f"{col}_{table_name}"
        #     for col in df_sub.columns
        #     if col in dupe_cols
        # }
        # if rename_map:
        #     df_sub = df_sub.rename(columns=rename_map)

        pieces.append(df_sub)

    file_df = pd.concat(pieces, axis=1) # master df for each file

    file_df.insert(0, "DevCo", [suffix] * len(file_df)) # append DevCo column

    file_frames.append(file_df) # append master df for each file to a list

master_df = pd.concat(file_frames, axis=0, ignore_index=True) # stack master df for each file row-wise

print("Final master shape:", master_df.shape)
# master_df


Reading 'assessment_guide_diriyah.xlsm' (suffix = diriyah)
⚠️ In table 'AddDataPoint(D&C)', missing columns: {nan}
Reading 'assessment_guide_qiddiya.xlsm' (suffix = qiddiya)
⚠️ In table 'AddDataPoint(D&C)', missing columns: {nan}
Final master shape: (156, 81)


In [86]:
for df in file_frames:
    dups = df.columns[df.columns.duplicated()].unique()
    if len(dups):
        print("⚠️ Duplicated column names in this frame:", dups.tolist())

In [87]:
output_path = repo_root/ 'master_sheet.xlsx'
master_df.to_excel(output_path, index=False)

print(f"✅ Saved master sheet to {output_path!r}")

✅ Saved master sheet to WindowsPath('c:/Users/jeremy.yap/OneDrive - Meinhardt Singapore Pte Ltd/Python/lighthouse_dash/master_sheet.xlsx')
